# DAL Framework Test

This notebook demonstrates and tests the Decentralized Active Learning framework.

Components tested:
1. Wallet Authentication
2. Active Learning
3. Voting System
4. Integration

In [1]:
# Import required packages
import numpy as np
from sklearn.datasets import make_classification
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
import ipywidgets as widgets

# Import DAL components
from DAL.backend.auth.wallet_auth import WalletAuth
from DAL.backend.blockchain_adapter import BlockchainAdapter
from DAL.backend.blockchain.voting_contract import VotingContract
from DAL.backend.al_manager import ActiveLearningManager

# Set random seed for reproducibility
np.random.seed(42)

ModuleNotFoundError: No module named 'DAL'

## 1. Create Synthetic Dataset

We'll create a synthetic binary classification dataset for testing.

In [1]:
# Create synthetic dataset
X, y = make_classification(
    n_samples=1000,
    n_features=2,  # 2D for easy visualization
    n_redundant=0,
    n_informative=2,
    random_state=42,
    n_clusters_per_class=1
)

# Split into initial labeled set and pool
n_initial = 10
indices = np.random.permutation(len(X))
X_initial = X[indices[:n_initial]]
y_initial = y[indices[:n_initial]]
X_pool = X[indices[n_initial:]]
y_pool = y[indices[n_initial:]]  # Ground truth (not used in real scenario)

# Plot dataset
plt.figure(figsize=(10, 6))
plt.scatter(X_pool[:, 0], X_pool[:, 1], c='gray', alpha=0.5, label='Unlabeled')
plt.scatter(X_initial[:, 0], X_initial[:, 1], c=y_initial, cmap='coolwarm', label='Initial')
plt.title('Synthetic Dataset')
plt.legend()
plt.show()

NameError: name 'make_classification' is not defined

## 2. Initialize Components

Set up the blockchain, authentication, and active learning components.

In [ ]:
# Initialize components
blockchain = BlockchainAdapter()
wallet_auth = WalletAuth(blockchain)
voting_contract = VotingContract(blockchain)
al_manager = ActiveLearningManager()

# Create test wallets for voters
voters = [wallet_auth.create_wallet() for _ in range(3)]
print("Created test wallets:")
for i, wallet in enumerate(voters):
    print(f"Voter {i+1}: {wallet['address']}")

## 3. Initialize Active Learning Model

In [ ]:
# Initialize the active learning model
estimator = SVC(probability=True, random_state=42)
al_manager.initialize_model(estimator, X_initial, y_initial)

# Query instances
n_query = 5
query_indices, uncertainty_scores = al_manager.query(X_pool, n_instances=n_query)

# Plot queried instances
plt.figure(figsize=(10, 6))
plt.scatter(X_pool[:, 0], X_pool[:, 1], c='gray', alpha=0.5, label='Unlabeled')
plt.scatter(X_initial[:, 0], X_initial[:, 1], c=y_initial, cmap='coolwarm', label='Initial')
plt.scatter(X_pool[query_indices, 0], X_pool[query_indices, 1], 
           c='yellow', s=100, marker='*', label='Queried')
plt.title('Active Learning Query')
plt.legend()
plt.show()

## 4. Create Voting Sessions

In [ ]:
# Create voting sessions for queried instances
session_ids = []
for idx, uncertainty in zip(query_indices, uncertainty_scores):
    sample_data = {
        "instance_id": int(idx),
        "features": X_pool[idx].tolist(),
        "uncertainty": float(uncertainty)
    }
    
    session_id = await voting_contract.create_voting_session(
        sample=sample_data,
        options=[0, 1]  # Binary classification
    )
    session_ids.append(session_id)
    print(f"Created voting session {session_id} for instance {idx}")

## 5. Simulate Voting

In a real scenario, this would be done through a UI by actual voters.

In [ ]:
# Simulate voting
for session_id in session_ids:
    for voter in voters:
        # In reality, this would be a human decision
        # Here we use ground truth with some noise
        idx = voting_contract.sessions[session_id].sample["instance_id"]
        true_label = y_pool[idx]
        
        # 80% chance of correct vote, 20% chance of incorrect
        if np.random.random() < 0.8:
            vote = true_label
        else:
            vote = 1 - true_label
            
        await voting_contract.cast_vote(
            session_id,
            voter["address"],
            vote
        )
        print(f"Voter {voter['address']} cast vote {vote} for session {session_id}")

## 6. Finalize Voting and Update Model

In [ ]:
# Finalize voting and collect results
voted_labels = []
for session_id in session_ids:
    outcome = await voting_contract.finalize_voting(session_id)
    voted_labels.append(outcome)
    print(f"Session {session_id} finalized with outcome {outcome}")

# Update model with new labels
X_update = X_pool[query_indices]
update_metrics = al_manager.update(X_update, np.array(voted_labels))
print(f"\nModel updated with performance: {update_metrics['performance']:.3f}")

# Plot final state
plt.figure(figsize=(10, 6))
plt.scatter(X_pool[:, 0], X_pool[:, 1], c='gray', alpha=0.5, label='Unlabeled')
plt.scatter(X_initial[:, 0], X_initial[:, 1], c=y_initial, cmap='coolwarm', label='Initial')
plt.scatter(X_update[:, 0], X_update[:, 1], c=voted_labels, 
           cmap='coolwarm', s=100, marker='*', label='Newly Labeled')
plt.title('Updated Dataset')
plt.legend()
plt.show()